In [47]:
#Carga de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

Carga de los datos

In [48]:
#CARGA DEL DATASET
data = pd.read_csv("game_info.csv", sep=',')

Primer vistaso a las columnas

In [49]:
pd.options.display.max_columns = 60
pd.options.display.max_columns

60

In [50]:
data.head(5)

,game_id,game_date,attendance,game_time
0,24600001,1946-11-01 00:00:00,NaN,NaN
1,24600003,1946-11-02 00:00:00,NaN,NaN
2,24600002,1946-11-02 00:00:00,NaN,NaN
3,24600004,1946-11-02 00:00:00,NaN,NaN
4,24600005,1946-11-02 00:00:00,NaN,NaN


Se comprueba que el dataset cargado tiene la cantidad de registros y columnas especificadas.

In [51]:
np.shape(data)

(58053, 4)

Se procedera a establecer un filtrado por fechas. Esto es para utilizar las temporadas siguientes (2020/2021) (2021/2022) (2022/2023) como fijan los parámetros del proyecto.

Dado que el final de la temporada anterior se retrasó debido la pandemia COVID-19, la NBA aprobó una fecha excepcional de inicio para el 22 de diciembre de 2020, cuando lo habitual era comenzar en el mes de octubre.

In [52]:
# Filtra los datos en fechas mayores a 2021-01-01.
data = data[data["game_date"]>'2020-12-31']
data = data[data["game_date"]<'2023-6-13']
np.shape(data)

(3527, 4)

In [53]:
data.tail(5)

,game_id,game_date,attendance,game_time
58048,42200403,2023-06-07 00:00:00,20019.0,2:24
58049,42200404,2023-06-09 00:00:00,20184.0,2:29
58050,42200405,2023-06-12 00:00:00,19537.0,2:22
58051,32200001,2023-02-19 00:00:00,17886.0,0:00
58052,32200001,2023-02-19 00:00:00,17886.0,0:00


In [54]:
data = data.drop(columns=['game_time'])

No es una columna relevante para el análisis asi que la dropeamos.

### Se consultan valores nulos por primera vez.

In [55]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0          0.000000
game_date               0          0.000000
attendance            535         15.168699


Observamos que tenemos un 15% de valores faltantes en la columna 'attendance'.

A continuación se fijara el parámetro por el cual se van a tratar los datos faltantes.

In [56]:
data['attendance'] = data['attendance'].fillna(data['attendance'].mean())

Se rellenaron los valores faltantes con la media de datos. 

Se consultan valores nulos nuevamente:

In [57]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0               0.0
game_date               0               0.0
attendance              0               0.0


Controlamos que no existan más datos duplicados.

In [58]:
duplicados = data.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = data[duplicados]
print(datos_duplicados)

        game_id            game_date  attendance
58051  32200001  2023-02-19 00:00:00     17886.0
58052  32200001  2023-02-19 00:00:00     17886.0


Observamos que hay un valor duplicado, procedemos a eliminarlo mediante su indice.

In [59]:
data = data.drop(58052)

Ahora que no tenemos valores faltantes ni duplicados, procedemos a verificar el formato de las columnas.

In [60]:
print(data.dtypes)

game_id         int64
game_date      object
attendance    float64
dtype: object


Cambiamos el formato de la tabla de fechas a 'datetime' debido a que esta en 'object'.

In [61]:
data['game_date'] = pd.to_datetime(data['game_date'], errors='coerce')

Revisamos los cambios realizados:

In [62]:
print(data['game_date'].dtypes)

datetime64[ns]


In [63]:
data.head()

,game_id,game_date,attendance
54483,22000063,2020-12-31,15353.935829
54484,22000068,2020-12-31,15353.935829
54485,22000065,2020-12-31,15353.935829
54486,22000064,2020-12-31,15353.935829
54487,22000067,2020-12-31,15353.935829


In [64]:
np.shape(data)

(3526, 3)

In [65]:
data.dtypes

game_id                int64
game_date     datetime64[ns]
attendance           float64
dtype: object

In [66]:
# Convierte la columna 'attendance' a int
data['attendance'] = data['attendance'].astype(int)

In [67]:
data.dtypes

game_id                int64
game_date     datetime64[ns]
attendance             int64
dtype: object

In [68]:
data

,game_id,game_date,attendance
54483,22000063,2020-12-31,15353
54484,22000068,2020-12-31,15353
54485,22000065,2020-12-31,15353
54486,22000064,2020-12-31,15353
54487,22000067,2020-12-31,15353
...,...,...,...
58047,42200402,2023-06-04,19537
58048,42200403,2023-06-07,20019
58049,42200404,2023-06-09,20184
58050,42200405,2023-06-12,19537


### Ahora exportamos el dataset ya procesado.

In [69]:
data.to_csv('game_info_filtrado.csv', index=False)